In [0]:
google_colab = True

# Google Colab用事前準備

In [2]:
# install
if google_colab:
  !pip install pybind11
  %cd /content
  !nvcc -V
  !sudo rm -fr BinaryBrain
  !rm -fr BinaryBrain
  !git clone --recursive -b ver3_develop https://github.com/ryuz/BinaryBrain.git
  %cd /content/BinaryBrain/python
  !sh copy_src.sh
  !python3 setup.py build
  !python3 setup.py develop

  import binarybrain as bb

     |████████████████████████████████| 153kB 6.7MB/s 
/content
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
Cloning into 'BinaryBrain'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 12626 (delta 193), reused 213 (delta 110), pack-reused 12294
Receiving objects: 100% (12626/12626), 5.63 MiB | 11.02 MiB/s, done.
Resolving deltas: 100% (9297/9297), done.
Submodule 'cereal' (https://github.com/USCiLab/cereal) registered for path 'cereal'
Submodule 'jelly' (https://github.com/ryuz/jelly) registered for path 'jelly'
Cloning into '/content/BinaryBrain/cereal'...
remote: Enumerating objects: 6, done.        
remote: Counting objects: 100% (6/6), done.        
remote: Compressing objects: 100% (5/5), done.        
remote: Total 18297 (delta 0), reused 2 (delta 0), pa

In [0]:
import binarybrain as bb

In [4]:
# mount
if google_colab:
  from google.colab import drive 
  drive.mount('/content/drive')
  %cd /content/drive/My Drive/git-work/BinaryBrain_ver3_develop/tests/python

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/git-work/BinaryBrain_ver3_develop/tests/python


In [5]:
import binarybrain as bb
prop = bb.get_device_properties()
print(prop)


name                     : Tesla P100-PCIE-16GB
totalGlobalMem           : 17071734784
sharedMemPerBlock        : 49152
regsPerBlock             : 65536
warpSize                 : 32
memPitch                 : 2147483647
maxThreadsPerBlock       : 1024
maxThreadsDim[0]         : 1024
maxThreadsDim[1]         : 1024
maxThreadsDim[2]         : 64
maxGridSize[0]           : 2147483647
maxGridSize[1]           : 65535
maxGridSize[2]           : 65535
clockRate                : 1328500
totalConstMem            : 65536
major                    : 6
minor                    : 0
textureAlignment         : 512
deviceOverlap            : 1
multiProcessorCount      : 56
kernelExecTimeoutEnabled : 0
integrated               : 0
canMapHostMemory         : 1
computeMode              : 0




# メインコード

In [0]:
import binarybrain as bb
import numpy as np
import os
import sys
from tqdm import tqdm
from collections import OrderedDict

In [7]:
# load MNIST data
td = bb.load_mnist()

dwonload /root/.binarybrain/dataset/train-images-idx3-ubyte.gz from http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
dwonload /root/.binarybrain/dataset/train-labels-idx1-ubyte.gz from http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
dwonload /root/.binarybrain/dataset/t10k-images-idx3-ubyte.gz from http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
dwonload /root/.binarybrain/dataset/t10k-labels-idx1-ubyte.gz from http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz


## 元レイヤー

In [8]:
# create layer
layer0_affine  = bb.DenseAffine.create([32])
layer0_norm    = bb.BatchNormalization.create()
layer0_bin     = bb.Binarize.create()
layer1_affine  = bb.DenseAffine.create([32])
layer1_norm    = bb.BatchNormalization.create()
layer1_bin     = bb.Binarize.create()
layer2_affine  = bb.DenseAffine.create([64])
layer2_norm    = bb.BatchNormalization.create()
layer2_bin     = bb.Binarize.create()
layer3_affine  = bb.DenseAffine.create([64])
layer3_norm    = bb.BatchNormalization.create()
layer3_bin     = bb.Binarize.create()
layer4_affine  = bb.DenseAffine.create([512])
layer4_norm    = bb.BatchNormalization.create()
layer4_bin     = bb.Binarize.create()
layer5_affine  = bb.DenseAffine.create([10])
layer5_norm    = bb.BatchNormalization.create()
layer5_bin     = bb.Binarize.create()

# main network
cnv0_sub = bb.Sequential.create()
cnv0_sub.add(layer0_affine)
cnv0_sub.add(layer0_norm)
cnv0_sub.add(layer0_bin)
layer0_cnv = bb.LoweringConvolution.create(cnv0_sub, 3, 3)

cnv1_sub = bb.Sequential.create()
cnv1_sub.add(layer1_affine)
cnv1_sub.add(layer1_norm)
cnv1_sub.add(layer1_bin)
layer1_cnv = bb.LoweringConvolution.create(cnv1_sub, 3, 3)

cnv2_sub = bb.Sequential.create()
cnv2_sub.add(layer2_affine)
cnv2_sub.add(layer2_norm)
cnv2_sub.add(layer2_bin)
layer2_cnv = bb.LoweringConvolution.create(cnv2_sub, 3, 3)

cnv3_sub = bb.Sequential.create()
cnv3_sub.add(layer3_affine)
cnv3_sub.add(layer3_norm)
cnv3_sub.add(layer3_bin)
layer3_cnv = bb.LoweringConvolution.create(cnv3_sub, 3, 3)

main_net = bb.Sequential.create()
main_net.add(layer0_cnv)
main_net.add(layer1_cnv)
main_net.add(bb.MaxPooling.create(2, 2))
main_net.add(layer2_cnv)
main_net.add(layer3_cnv)
main_net.add(bb.MaxPooling.create(2, 2))
main_net.add(layer4_affine)
main_net.add(layer4_norm)
main_net.add(layer4_bin)
main_net.add(layer5_affine)
main_net.add(layer5_norm)
main_net.add(layer5_bin)

main_net.set_input_shape(td['x_shape'])

# Load
net_path = 'mnist-dense-cnn-binary'

layer0_affine.load_json(os.path.join(net_path, 'layer0_affine.json'))
layer1_affine.load_json(os.path.join(net_path, 'layer1_affine.json'))
layer2_affine.load_json(os.path.join(net_path, 'layer2_affine.json'))
layer3_affine.load_json(os.path.join(net_path, 'layer3_affine.json'))
layer4_affine.load_json(os.path.join(net_path, 'layer4_affine.json'))
layer5_affine.load_json(os.path.join(net_path, 'layer5_affine.json'))

layer0_norm.load_json(os.path.join(net_path, 'layer0_norm.json'))
layer1_norm.load_json(os.path.join(net_path, 'layer1_norm.json'))
layer2_norm.load_json(os.path.join(net_path, 'layer2_norm.json'))
layer3_norm.load_json(os.path.join(net_path, 'layer3_norm.json'))
layer4_norm.load_json(os.path.join(net_path, 'layer4_norm.json'))
layer5_norm.load_json(os.path.join(net_path, 'layer5_norm.json'))

loss      = bb.LossSoftmaxCrossEntropy.create()
metrics   = bb.MetricsCategoricalAccuracy.create()
optimizer = bb.OptimizerAdam.create()
optimizer.set_variables(main_net.get_parameters(), main_net.get_gradients())

runner = bb.Runner(main_net, "mnist-dense-cnn-binary", loss, metrics, optimizer)
runner.fitting(td, epoch_size=1, mini_batch_size=32, file_write=False, file_read=False)

epoch=1 test_accuracy=0.985100 test_loss=1.468799 train_accuracy=0.988883 train_loss=1.466212


## 蒸留先

In [9]:
layer_cnv0_sl0 = bb.SparseLut6.create([192])
layer_cnv0_sl1 = bb.SparseLut6.create([32])

layer_cnv1_sl0 = bb.SparseLut6.create([192])
layer_cnv1_sl1 = bb.SparseLut6.create([32])

layer_cnv2_sl0 = bb.SparseLut6.create([384])
layer_cnv2_sl1 = bb.SparseLut6.create([64])

layer_cnv3_sl0 = bb.SparseLut6.create([384])
layer_cnv3_sl1 = bb.SparseLut6.create([64])

layer_sl4_0    = bb.SparseLut6.create([512*6])
layer_sl4_1    = bb.SparseLut6.create([512])
layer_sl5_0    = bb.SparseLut6.create([10*6*6])
layer_sl5_1    = bb.SparseLut6.create([10*6])
layer_sl5_2    = bb.SparseLut6.create([10])

# main network
target_cnv0_sub = bb.Sequential.create()
target_cnv0_sub.add(layer_cnv0_sl0)
target_cnv0_sub.add(layer_cnv0_sl1)
target_layer0_cnv = bb.LoweringConvolution.create(target_cnv0_sub, 3, 3)

target_cnv1_sub = bb.Sequential.create()
target_cnv1_sub.add(layer_cnv1_sl0)
target_cnv1_sub.add(layer_cnv1_sl1)
target_layer1_cnv = bb.LoweringConvolution.create(target_cnv1_sub, 3, 3)

target_cnv2_sub = bb.Sequential.create()
target_cnv2_sub.add(layer_cnv2_sl0)
target_cnv2_sub.add(layer_cnv2_sl1)
target_layer2_cnv = bb.LoweringConvolution.create(target_cnv2_sub, 3, 3)

target_cnv3_sub = bb.Sequential.create()
target_cnv3_sub.add(layer_cnv3_sl0)
target_cnv3_sub.add(layer_cnv3_sl1)
target_layer3_cnv = bb.LoweringConvolution.create(target_cnv3_sub, 3, 3)

target_layer4 = bb.Sequential.create()
target_layer4.add(layer_sl4_0)
target_layer4.add(layer_sl4_1)

target_layer5 = bb.Sequential.create()
target_layer5.add(layer_sl5_0)
target_layer5.add(layer_sl5_1)
target_layer5.add(layer_sl5_2)

target_net = bb.Sequential.create()
target_net.add(target_layer0_cnv)
target_net.add(target_layer1_cnv)
target_net.add(bb.MaxPooling.create(2, 2))
target_net.add(target_layer0_cnv)
target_net.add(target_layer1_cnv)
target_net.add(bb.MaxPooling.create(2, 2))
target_net.add(target_layer4)
target_net.add(target_layer5)

target_net.set_input_shape(td['x_shape'])

main_net = bb.Sequential.create()
main_net.add(layer0_cnv)
main_net.add(layer1_cnv)
main_net.add(bb.MaxPooling.create(2, 2))
main_net.add(layer2_cnv)
main_net.add(layer3_cnv)
main_net.add(bb.MaxPooling.create(2, 2))
main_net.add(layer4_affine)
main_net.add(layer4_norm)
main_net.add(layer4_bin)
main_net.add(layer5_affine)
main_net.add(layer5_norm)
main_net.add(layer5_bin)

main_net.set_input_shape(td['x_shape'])

[10]

In [0]:
# 1レイヤーの蒸留
def layer_distillation(x, x_shape, target_net, ref_net, pre_net, epoch=8):
    x_buf = bb.FrameBuffer()
    t_buf = bb.FrameBuffer()

    print(ref_net.get_input_shape())

    target_net.set_input_shape(ref_net.get_input_shape())
    target_net.send_command("binary true")
    
    pre_net.set_input_shape(x_shape)

    batch_size = len(x)
    max_batch_size = 32
    leave = True

    loss = bb.LossMeanSquaredError.create()
    optimizer = bb.OptimizerAdam.create()
    optimizer.set_variables(target_net.get_parameters(), target_net.get_gradients())
    
    for e in range(epoch):
        loss.clear()
        with tqdm(range(0, batch_size, max_batch_size), leave=leave) as pbar:
            for index in pbar:
                # calc mini_batch_size
                mini_batch_size = min(max_batch_size, batch_size-index)
                
                # setup x
                x_buf.resize(mini_batch_size, x_shape)
                x_buf.set_data(x[index:index+mini_batch_size])
                
                # forward
                x_buf = pre_net.forward(x_buf, False)
                t_buf = ref_net.forward(x_buf, False)
                y_buf = target_net.forward(x_buf, True)
                
                # calc loss
                dy_buf = loss.calculate_loss(y_buf, t_buf, mini_batch_size)

                # backward
                target_net.backward(dy_buf)

                # update
                optimizer.update()
                
                # print progress
                dict = OrderedDict()
                dict['loss'] = loss.get_loss()
                if len(dict) > 0:
                    pbar.set_postfix(dict)

In [0]:
# 事前ネット
pre_net = bb.Sequential.create()
pre_net.add(bb.RealToBinary.create(8, framewise=True))

In [12]:
# Layer0
tW = layer0_affine.W()
W = np.array(tW.get_data()).reshape(tW.get_shape()[::-1])
idx = np.argsort(-np.abs(W), axis=1)

# Weight順で接続
if True:
    for i in range(32):
        for j in range(6):
            layer_cnv0_sl1.set_connection_index([i], j, i*6+j)

    for i in range(32):
        for j in range(6*6):
            layer_cnv0_sl0.set_connection_index([i], j, idx[i][j % 3*3])


layer_cnv0_sl0.load_json(os.path.join(net_path, 'layer_cnv0_sl0.json'))
layer_cnv0_sl1.load_json(os.path.join(net_path, 'layer_cnv0_sl1.json'))

layer_distillation(td['x_train'], td['x_shape'], target_layer0_cnv, layer0_cnv, pre_net, epoch=1)

  0%|          | 0/1875 [00:00<?, ?it/s]

[28, 28, 1]


100%|██████████| 1875/1875 [44:28<00:00,  1.35s/it, loss=0.0686]


In [0]:
layer_cnv0_sl0.save_json(os.path.join(net_path, 'layer_cnv0_sl0.json'))
layer_cnv0_sl1.save_json(os.path.join(net_path, 'layer_cnv0_sl1.json'))



In [0]:
# Layer1
tW = layer1_affine.W()
W = np.array(tW.get_data()).reshape(tW.get_shape()[::-1])
idx = np.argsort(-np.abs(W), axis=1)

In [0]:
if True:
    for i in range(32):
        for j in range(6):
            layer_cnv1_sl1.set_connection_index([i], j, i*6+j)

    for i in range(32):
        for j in range(6*6):
            layer_cnv1_sl0.set_connection_index([i], j, idx[i][j])

In [0]:
if False:
    for i in range(32):
        for j in range(6):
            layer_cnv1_sl3.set_connection_index([i], j, i*6+j)
    
    for i in range(32):
        for j in range(6*6):
            layer_cnv1_sl2.set_connection_index([i], j, i*6*6+j)
            
    for i in range(32):
        for j in range(6*6):
            layer_cnv1_sl1.set_connection_index([i], j, i*6*6+j)

    for i in range(32):
        for j in range(6*6*6):
#            if j < len(idx[i]):
            layer_cnv1_sl0.set_connection_index([i], j, idx[i][j % len(idx[i])])

In [0]:
tW = layer1_affine.W()
W = np.array(tW.get_data()).reshape(tW.get_shape()[::-1])

In [0]:
idx = np.argsort(-np.abs(W), axis=1)

In [0]:
pre_net = bb.Sequential.create()
pre_net.add(bb.RealToBinary.create(4, framewise=True))

In [0]:
pre_net.add(layer0_cnv)

In [21]:
layer_distillation(td['x_train'], td['x_shape'], target_layer1_cnv, layer1_cnv, pre_net)

  0%|          | 0/1875 [00:00<?, ?it/s]

[26, 26, 32]


100%|██████████| 1875/1875 [11:04<00:00,  2.80it/s, loss=0.323]


In [0]:
#layer_cnv1_sl0.save_json(os.path.join(net_path, 'layer1_cnv_sl0.json'))
#layer_cnv1_sl1.save_json(os.path.join(net_path, 'layer1_cnv_sl1.json'))

In [0]:
layer_cnv1_sl0 = bb.SparseLut6.create([192])
layer_cnv1_sl1 = bb.SparseLut6.create([32])


In [24]:
layer_cnv1_sl1.set_input_shape([192])
#layer_cnv1_sl1.set_connection_index([0], 0, 0)

[32]